In [1]:
# ToDo: Rerun the code when the missing data is available
import collections
import numpy as np
import pandas as pd
import re

from argparse import Namespace

In [2]:
args = Namespace(
    raw_train_dataset_csv="data/yelp/raw_train.csv",
    raw_test_dataset_csv="data/yelp/raw_test.csv",
    train_proportion=0.7,
    val_proportion=0.3,
    output_munged_csv="data/yelp/reviews_with_splits_full.csv",
    seed=1337
)

In [3]:
# Read raw data
train_reviews = pd.read_csv(args.raw_train_dataset_csv, header=None, names=['rating', 'review'])
train_reviews = train_reviews[~pd.isnull(train_reviews.review)]
test_reviews = pd.read_csv(args.raw_test_dataset_csv, header=None, names=['rating', 'review'])
test_reviews = test_reviews[~pd.isnull(test_reviews.review)]

In [4]:
train_reviews.head()

,,,rating,review
"<html lang=""en"" dir=ltr><meta charset=utf-8><meta name=viewport content=""initial-scale=1",minimum-scale=1,"width=device-width""><title>Error 404 (Not Found)!!1</title><style nonce=""MPORc_7ltneSYnCiaUg4yQ"">*{margin:0;padding:0}html",code{font:15px/22px arial,sans-serif}html{background:#fff;color:#222;pad...


In [5]:
test_reviews.head()

,rating,review
0,1,Ordered a large Mango-Pineapple smoothie. Stay...
1,2,Quite a surprise! \n\nMy wife and I loved thi...
2,1,"First I will say, this is a nice atmosphere an..."
3,2,I was overall pretty impressed by this hotel. ...
4,1,Video link at bottom review. Worst service I h...


In [6]:
# Unique classes
set(train_reviews.rating)

{'code{font:15px/22px arial'}

In [7]:
# Splitting train by rating
# Create dict
by_rating = collections.defaultdict(list)
for _, row in train_reviews.iterrows():
    by_rating[row.rating].append(row.to_dict())

In [8]:
# Create split data
final_list = []
np.random.seed(args.seed)

for _, item_list in sorted(by_rating.items()):

    np.random.shuffle(item_list)

    n_total = len(item_list)
    n_train = int(args.train_proportion * n_total)
    n_val = int(args.val_proportion * n_total)

    # Give data point a split attribute
    for item in item_list[:n_train]:
        item['split'] = 'train'

    for item in item_list[n_train:n_train+n_val]:
        item['split'] = 'val'

    # Add to final list
    final_list.extend(item_list)

In [9]:
for _, row in test_reviews.iterrows():
    row_dict = row.to_dict()
    row_dict['split'] = 'test'
    final_list.append(row_dict)

In [10]:
# Write split data to file
final_reviews = pd.DataFrame(final_list)

In [11]:
final_reviews.split.value_counts()

split
test    38000
Name: count, dtype: int64

In [12]:
final_reviews.review.head()

0    sans-serif}html{background:#fff;color:#222;pad...
1    Ordered a large Mango-Pineapple smoothie. Stay...
2    Quite a surprise!  \n\nMy wife and I loved thi...
3    First I will say, this is a nice atmosphere an...
4    I was overall pretty impressed by this hotel. ...
Name: review, dtype: object

In [13]:
final_reviews[pd.isnull(final_reviews.review)]

,rating,review,split


In [14]:
# Preprocess the reviews
def preprocess_text(text):
    if type(text) == float:
        print(text)
    text = text.lower()
    text = re.sub(r"([.,!?])", r" \1 ", text)
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)
    return text

final_reviews.review = final_reviews.review.apply(preprocess_text)

In [15]:
final_reviews['rating'] = final_reviews.rating.apply({1: 'negative', 2: 'positive'}.get)

In [16]:
final_reviews.head()

,rating,review,split
0,None,sans serif html background fff color padding p...,NaN
1,negative,ordered a large mango pineapple smoothie . sta...,test
2,positive,quite a surprise ! n nmy wife and i loved this...,test
3,negative,"first i will say , this is a nice atmosphere a...",test
4,positive,i was overall pretty impressed by this hotel ....,test


In [17]:
final_reviews.to_csv(args.output_munged_csv, index=False)